<a href="https://colab.research.google.com/github/JatinKrRana/AI-job-recommendation-web-app/blob/main/job_recommendation_system_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 AI-Powered Job Recommendation System using LangChain & Gemini

## 📌 Introduction
This notebook contains the initial code exploration and testing for the AI Job Recommendation Web App.
We experiment with:
- LangChain prompt templates
- Gemini API integration
- Job dataset loading
- RAG (Retrieval-Augmented Generation) pipeline setup


In [ ]:
!pip install langchain-groq langchain python-dotenv pdfplumber python-docx serpapi
!pip install streamlit

In [ ]:
!pip uninstall -y serpapi google-search-results
!pip install google-search-results==2.4.2


In [3]:
import os
import pdfplumber
import docx
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate


In [4]:
#Load Environment Variables
os.environ["GROQ_API_KEY"] = "your api key"
os.environ["SERPAPI_API_KEY"] = "your api key"


In [ ]:
# Testing LangChain + Gemini
llm = ChatGroq(model="llama3-8b-8192", temperature=0)

prompt_template = PromptTemplate(
    input_variables=["skills", "experience", "location", "job_type"],
    template="""
You are a job recommendation assistant. Based on the details below, suggest 5 suitable jobs with reasons:

Skills: {skills}
Experience: {experience}
Location: {location}
Job Type: {job_type}
    """
)

final_prompt = prompt_template.format(
    skills="Python, SQL, Data Analysis",
    experience="1 year internship in analytics",
    location="India",
    job_type="Internship"
)

response = llm.invoke(final_prompt)
print(response.content)


In [ ]:
# Testing Job Scraper (Google Jobs via SerpAPI)
from serpapi import GoogleSearch

def get_google_jobs(query, location="India", job_type=None):
    params = {
        "engine": "google_jobs",
        "q": query,
        "location": location,
        "api_key": os.environ["SERPAPI_API_KEY"]
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    return results.get("jobs_results", [])

jobs = get_google_jobs("Data Analyst Internship", "India")
jobs[:2]  # show first two results


In [ ]:
# Mini Retrieval-Augmented Generation Test

# Step 1: Retrieve jobs (dummy data if SerpAPI isn't available)
jobs = [
    {"title": "Data Analyst Intern", "company_name": "ABC Corp", "location": "Bangalore"},
    {"title": "Business Intelligence Intern", "company_name": "XYZ Pvt Ltd", "location": "Mumbai"},
    {"title": "Python Developer Intern", "company_name": "Tech Solutions", "location": "Delhi"},
    {"title": "Data Science Intern", "company_name": "AI Labs", "location": "Pune"},
    {"title": "SQL Developer Intern", "company_name": "DataWorks", "location": "Hyderabad"}
]

# Step 2: Format job data into plain text
jobs_text = "\n".join([f"{job['title']} at {job['company_name']} ({job['location']})" for job in jobs])

# Step 3: Create a prompt for the LLM
rag_prompt = PromptTemplate(
    input_variables=["skills", "jobs"],
    template="""
You are an AI assistant recommending jobs.
Given these skills: {skills}
And these job postings:
{jobs}

Select the top 3 most suitable jobs and explain why they match.
"""
)

# Step 4: Run the LLM with the prompt
formatted_prompt = rag_prompt.format(
    skills="Python, SQL, Data Analysis",
    jobs=jobs_text
)

rag_response = llm.invoke(formatted_prompt)

# Step 5: Display the AI's recommendations
print(rag_response.content)


## 📌 Conclusion
- Successfully tested LangChain prompt template
- Integrated Gemini (via Groq) for job recommendations
- Fetched real job postings using SerpAPI
- Verified end-to-end RAG pipeline before integrating into Streamlit app
